In [5]:
import math
import time
import psycopg2
import pandas as pd
import numpy as np
from datetime import datetime
from scipy.sparse import csr_matrix
from fast_pagerank import pagerank
from datetime import datetime, timedelta

In [7]:
n = 0.6 # min confidence; 1/search_cnt^n
min_lift = 1
confidence_weight = 0.7 
lift_weight = 0.3
initial_date = "{:%Y%m%d}".format(datetime.now() - timedelta(days=6))
final_date = "{:%Y%m%d}".format(datetime.now() - timedelta(days=1))

top_k = 20

*밑 쿼리는 이미 실행했으니 실행 안해도됨

In [ ]:
q = f'''
DROP TABLE IF EXISTS workspace.search_term_log_modified;


CREATE TABLE workspace.search_term_log_modified AS
SELECT user_id,
       session_id,
       lower(trim(regexp_replace(regexp_replace(search_term, '\\r|\\n', ' '), '\\s+', ' '))) AS keyword,
       LAG(keyword, 1) OVER (PARTITION BY user_id ORDER BY server_time_kst) AS prev_keyword,
       LEAD(keyword, 1) OVER (PARTITION BY user_id ORDER BY server_time_kst) AS next_keyword,
       REPLACE(search_term, ' ', '') AS keyword_no_space,
       REPLACE(prev_keyword, ' ', '') AS prev_keyword_no_space,
       REPLACE(next_keyword, ' ', '') AS next_keyword_no_space,
       server_time_kst::TIMESTAMP AS new_server_time,
       LAG(server_time_kst, 1) OVER (PARTITION BY user_id ORDER BY server_time_kst)::TIMESTAMP AS prev_server_time,
       LEAD(server_time_kst, 1) OVER (PARTITION BY user_id ORDER BY server_time_kst)::TIMESTAMP AS next_server_time,
       datediff(SECOND, prev_server_time, new_server_time) AS cur_prev_duration,
       datediff(SECOND, prev_server_time, next_server_time) AS prev_next_duration
FROM bun_log_db.app_event_type_search
WHERE YEAR || MONTH || DAY >= {initial_date}
  AND YEAR || MONTH || DAY < {final_date}
  AND search_term IS NOT NULL
  AND search_term != ''
  AND search_term != ' '
  AND user_id > 0
  AND device_type IN ('a',
                      'i');


DROP TABLE IF EXISTS workspace.keyword_view_count;


CREATE TABLE workspace.keyword_view_count AS
SELECT lower(trim(regexp_replace(regexp_replace(ref_term, '\\r|\\n', ' '), '\\s+', ' '))) AS keyword,
       COUNT(*) AS view_count
FROM bun_log_db.app_event_type_view v
WHERE YEAR || MONTH || DAY >= {initial_date}
  AND YEAR || MONTH || DAY < {final_date}
  AND event_action = 'view_content'
  AND content_type = 'product'
  AND v.ref_term IS NOT NULL
  AND v.ref_term != ''
  AND v.ref_term != ' '
  AND device_type IN ('a',
                      'i')
GROUP BY 1;


DROP TABLE IF EXISTS workspace.search_results_count;


CREATE TABLE workspace.search_results_count AS
SELECT lower(trim(regexp_replace(regexp_replace(search_term, '\\r|\\n', ' '), '\\s+', ' '))) AS keyword,
       MAX((search_results_count):: int) AS results_count
FROM bun_log_db.api_event_type_search
WHERE YEAR || MONTH || DAY >= {initial_date}
  AND YEAR || MONTH || DAY < {final_date}
  AND device_type IN ('a',
                      'i')
GROUP BY 1;


DROP TABLE IF EXISTS workspace.keyword_search_count;


CREATE TABLE workspace.keyword_search_count AS
SELECT keyword,
       count(*) AS search_count
FROM workspace.search_term_log_modified
GROUP BY 1;


DROP TABLE IF EXISTS workspace.related_keyword_prev_cur;


CREATE TABLE workspace.related_keyword_prev_cur AS
SELECT prev_keyword AS keyword,
       keyword AS related_keyword,
       COUNT(*) AS COUNT
FROM workspace.search_term_log_modified
WHERE prev_keyword IS NOT NULL
  AND cur_prev_duration <= 600
  AND cur_prev_duration >= 1
  AND keyword_no_space != prev_keyword_no_space
GROUP BY 1,
         2;


DROP TABLE IF EXISTS workspace.related_keyword_prev_next;


CREATE TABLE workspace.related_keyword_prev_next AS
SELECT prev_keyword AS keyword,
       next_keyword AS related_keyword,
       COUNT(*) AS COUNT
FROM workspace.search_term_log_modified
WHERE prev_keyword IS NOT NULL
  AND cur_prev_duration <= 600
  AND cur_prev_duration >= 1
  AND next_keyword_no_space != prev_keyword_no_space
GROUP BY 1,
         2;


DROP TABLE IF EXISTS workspace.related_keyword_combined;


CREATE TABLE workspace.related_keyword_combined AS
SELECT *
FROM
  (SELECT c.keyword,
          c.related_keyword,
          c.count + n.count AS related_count,

     (SELECT COUNT(*)
      FROM search_term_log_modified) AS total_search_count,
          sc.search_count,
          rsc.search_count AS related_search_count,
          related_count / total_search_count :: float AS support,
          related_count / sc.search_count :: float AS confidence,
          confidence / (rsc.search_count / total_search_count :: float) AS lift
   FROM workspace.related_keyword_prev_cur c
   JOIN workspace.related_keyword_prev_next n ON c.keyword = n.keyword
   AND c.related_keyword = n.related_keyword
   JOIN workspace.keyword_search_count sc ON c.keyword = sc.keyword
   JOIN workspace.keyword_search_count rsc ON c.related_keyword = rsc.keyword) a
WHERE related_count > 2;


DROP TABLE IF EXISTS workspace.related_keyword_df_temp;


CREATE TABLE workspace.related_keyword_df_temp AS
SELECT k.keyword,
       k.related_keyword,
       k.related_count,
       k.search_count,
       k.related_search_count,
       rc.results_count,
       v.view_count AS related_view_count,
       k.total_search_count,
       k.support,
       k.confidence,
       k.lift
FROM workspace.related_keyword_combined k
JOIN workspace.search_results_count rc ON k.related_keyword = rc.keyword
JOIN workspace.keyword_view_count v ON k.related_keyword = v.keyword
WHERE results_count > 1
ORDER BY 4 DESC,
         1,
         10 DESC;


DROP TABLE IF EXISTS workspace.related_keyword_df_filtered;


CREATE TABLE workspace.related_keyword_df_filtered AS
SELECT *
FROM workspace.related_keyword_df_temp
WHERE confidence > 1 / pow(search_count, {n})
  AND lift > {min_lift};


DROP TABLE IF EXISTS workspace.related_keyword_df_max;


CREATE TABLE workspace.related_keyword_df_max AS
SELECT keyword,
       max(confidence) AS max_confidence,
       max(lift) AS max_lift
FROM workspace.related_keyword_df_filtered
GROUP BY 1;


DROP TABLE IF EXISTS workspace.related_keyword_df;


CREATE TABLE workspace.related_keyword_df AS
SELECT a.keyword,
       a.related_keyword,
       a.related_count,
       a.search_count,
       ROUND(({confidence_weight} * a.confidence / b.max_confidence:: float) + ({lift_weight} * a.lift / b.max_lift:: float),
             4) AS score
FROM workspace.related_keyword_df_filtered a
JOIN workspace.related_keyword_df_max b ON a.keyword = b.keyword
ORDER BY a.search_count DESC,
         a.keyword,
         score DESC;'''
cur.execute(query)
conn.commit()
cur.close()
conn.close()

In [5]:
conn = psycopg2.connect(dbname='bunjang', host='172.31.133.162',
                       port='5439', user=user_id, password=pwd)

query = f"""
SELECT *
FROM workspace.related_keyword_df;
"""

data = pd.read_sql(query, conn)
conn.close()


In [32]:
data.head()

,keyword,related_keyword,related_count,search_count,score
0,자전거,로드자전거,368,17308,0.8029
1,자전거,mtb자전거,239,17308,0.5761
2,자전거,중고자전거,78,17308,0.4484
3,자전거,전기자전거,174,17308,0.3903
4,자전거,중고전기자전거,111,17308,0.3711


In [33]:
df = df[['keyword', 'related_keyword', 'related_count']]

In [17]:
# df['zip'] = [tuple(x) for x in df[['related_keyword', 'related_count']].values] 
# tmp = df.groupby('keyword')['zip'].apply(list).reset_index()

In [34]:
keyword_dict = {}
for idx, row in df.iterrows():
    tmp = keyword_dict.get(row.keyword, {})
    if row.related_keyword.replace(' ', '') not in [k.replace(' ', '') for k in tmp.keys()]:
        tmp[row.related_keyword] = row.related_count
        keyword_dict[row.keyword] = tmp

In [35]:
keyword_dict['자전거']

{'mtb': 155,
 'mtb자전거': 239,
 '로드': 74,
 '로드자전거': 368,
 '미니벨로': 100,
 '실내자전거': 50,
 '오토바이': 95,
 '전기자전거': 174,
 '전동킥보드': 76,
 '접이식자전거': 72,
 '중고자전거': 78,
 '중고전기자전거': 111,
 '픽시': 143,
 '하이브리드 자전거': 62}

In [58]:
def expand_related_keywords(keyword):
    ranking_list = []
    candidate_list = []
    for related_keyword, related_count in keyword_dict[keyword].items():
        ranking_list.append([keyword, related_keyword, related_count])
        if related_keyword in keyword_dict:
            candidate_list.append(related_keyword)
    i = 0
    while i < 2:
        new_candidate_list = []
        for candidate in candidate_list:
            for related_keyword, related_count in keyword_dict[candidate].items():
                if [candidate, related_keyword, related_count] not in ranking_list:
                    ranking_list.append([candidate, related_keyword, related_count])
                if related_keyword in keyword_dict:
                    new_candidate_list.append(related_keyword)
        candidate_list = new_candidate_list
        i += 1
        
    return ranking_list

In [59]:
final_dict = {}
for keyword, related_keyword_dict in keyword_dict.items():
    final_dict[keyword] = expand_related_keywords(keyword)

----------------------------------------------------------------------

In [60]:
def pageRank(keyword):
    related_keyword_list = final_dict[keyword]
    
    node_list = []
    for i in related_keyword_list:
        node_list += i[:2]
    node_list = sorted(list(set(node_list)))
    
    node_to_idx = {node: idx for idx, node in enumerate(node_list)}
    idx_to_node = {idx: node for idx, node in enumerate(node_list)}
    
    A = []
    related_counts = []
    for fr, to, related_count in related_keyword_list:
        A.append([node_to_idx[fr], node_to_idx[to]])
        related_counts.append(related_count)
    
    A = np.array(A)
    G = csr_matrix((related_counts, (A[:, 0], A[:, 1])), shape=(len(node_list), len(node_list)))
    pr = pagerank(G, p=0.85)
    
    rank = {idx_to_node[idx]: value for idx, value in enumerate(pr)}
    
    keyword_list = []
    for related_keyword, value in sorted(rank.items(), key=lambda x: x[1], reverse=True):
        if related_keyword.replace(' ', '') != keyword.replace(' ', '') and
        related_keyword.replace(' ', '') not in [k.replace(' ', '') for k in keyword_list] and
        len(keyword_list) < top_k:
            keyword_list.append(related_keyword)
    return keyword_list  

In [61]:
df_final = {}

for keyword in keyword_dict:
    tmp = pageRank(keyword)
    df_final[keyword] = tmp

In [62]:
column_name = []
for idx in range(top_k):
    column_name.append('relatd_keyword_{}'.format(idx+1))
search_count = data.groupby('keyword', as_index=False)['search_count'].mean()

In [63]:
temp_related_keyword_df = pd.DataFrame.from_dict(df_final, orient='index', columns = column_name)
related_keyword_df2 = pd.merge(search_count, temp_related_keyword_df, left_on = 'keyword', right_index=True).sort_values('search_count', ascending=False)

In [65]:
related_keyword_df2.to_csv('related_keyword_df2.csv', encoding='utf-8', index=False)

In [64]:
related_keyword_df2

,keyword,search_count,relatd_keyword_1,relatd_keyword_2,relatd_keyword_3,relatd_keyword_4,relatd_keyword_5,relatd_keyword_6,relatd_keyword_7,relatd_keyword_8,...,relatd_keyword_11,relatd_keyword_12,relatd_keyword_13,relatd_keyword_14,relatd_keyword_15,relatd_keyword_16,relatd_keyword_17,relatd_keyword_18,relatd_keyword_19,relatd_keyword_20
55502,자전거,17308,픽시,로드자전거,오토바이,전동킥보드,콘스탄틴,스쿠터,미니벨로,mtb자전거,...,메리다,자이언트,엔진11,mtb,bmx,언노운,실내자전거,전동스쿠터,스페셜라이즈드,pcx
44041,아이패드,16880,아이패드 에어4,에어팟 프로,아이패드 에어3,아이패드 8세대,아이패드프로4세대 12.9,에어팟,에어팟2,애플펜슬 2세대,...,갤럭시탭s6,애플워치,노트북,아이폰12,갤럭시탭s7,애플펜슬 1세대,아이폰11,아이패드 7세대,아이폰,아이패드 프로
11316,귀멸의칼날,15946,하이큐,주술회전,렌고쿠,귀멸의칼날 피규어,귀멸의칼날 만화책,가면라이더,귀멸의칼날 특전,하이큐 아크릴,...,애니 인스,나루토,진격의거인,하이큐 인스,시노부,이노스케,고르는 인스,렌고쿠 피규어,인스 띵,캐릭캐릭체인지
44426,아이폰,15730,아이폰xs,아이폰11,에어팟 프로,아이폰12,아이폰8,아이폰se2,아이폰x,아이폰7,...,에어팟2,에어팟,아이폰12미니,아이폰se,아이패드 에어4,아이폰11프로,아이패드 8세대,에어팟2 미개봉,에어팟 프로 미개봉,아이패드 에어3
15585,노트북,14252,아이패드,컴퓨터,아이패드 에어4,게이밍노트북,레노버,아이패드 8세대,아이패드 에어3,맥북,...,게이밍컴퓨터,asus,아이패드 7세대,아이패드 프로,한성,hp,아이패드 6세대,애플펜슬,아이패드 프로 4세대,gtx
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12447,꼼데 스트라이프티,2,네이더스 스트라이프,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
53767,이대휘 졸업앨범,2,이대휘,이대휘 포카,전웅,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
28565,미니어쳐 커피잔,2,미니어쳐 리모지,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
28567,미니어쳐 형수,2,미니어쳐 향수,향수 미니,향수 미니어처,다마고치,다마고치 썸,햄스터,실바니안,씰스티커,...,None,None,None,None,None,None,None,None,None,None
